# Постановка задачи

### Description
Участникам предоставлен набор данных, содержащий url и title веб-страниц, а также метку класса - 1, если страница относится к контенту 18+ и 0 - не относится к контенту 18+.

Задача состоит в том, чтобы на основе предоставленного набора тренировочных данных построить бинарный классификатор веб-страниц.

### Evaluation
Метрика качества в данном соревновании: F1-Score. F1-Score представляет собой среднее гармоническое между точностью - Precision и полнотой - Recall. Precision - отношение числа истинно-положительных предсказаний (true positives, tp) ко всем предсказанным положительным результатам (tp + fp). Recall- отношение истинно-положительных предсказаний ко всем положительным событиям набора данных (tp + fn).

Разбиение тестового набора на Public-/Private- части выполнено в пропорции 30/70%.

Файл ответов должен содержать заголовок и иметь следующий вид: 

ID,label 

135309,0 

135310,0 

135311,0 

135312,1 

...


### Dataset Description
Описание файлов

train.csv - тренировочный набор данных (поля: ID, url, title, label) 
test.csv - тестовый набор данных (поля: ID, url, title) 
baseline.ipynb - Jupyter-notebook с примером подготовки решения

Поля данных

ID - уникальный id страницы 
url - доменное имя страницы 
title - заголовок страницы 
label - целевая переменная (1, если страница относится к контенту 18+, и 0 в противном случае)

# Решение

In [350]:
!pip install numpy pandas emot nltk wordsegment optuna pymorphy3 pymorphy3-dicts-ru tqdm xgboost catboost scikit-learn matplotlib seaborn progressbar2

In [203]:
import numpy as np
import pandas as pd
import re
import warnings
import emot
import nltk
import wordsegment
import optuna
import pymorphy3
from progressbar import progressbar
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
from tqdm import notebook
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from collections import Counter
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import mean_squared_error
from optuna.samplers import TPESampler

In [210]:
max_features = 1000000
tfidf = TfidfVectorizer(max_features=max_features)

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
wordsegment.load()
stop_words = nltk.corpus.stopwords.words('russian')
TOKEN_PATTERN = "[а-яёa-z]+"

[nltk_data] Error loading stopwords: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>
[nltk_data] Error loading punkt: <urlopen error [Errno 8] nodename nor
[nltk_data]     servname provided, or not known>


## Обработка данных

In [3]:
train_df = pd.read_csv("train.csv")
train_df

,ID,url,title,label
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",0
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,0
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,0
3,3,colorbox.spb.ru,Не Беси Меня Картинки,0
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,0
...,...,...,...,...
135304,135304,mail.ru,пора тюльпанов турецкий сериал на русском язык...,0
135305,135305,www.ntv.ru,Остросюжетный сериал «Шеф. Игра на повышение»....,0
135306,135306,topclassiccarsforsale.com,"1941 Plymouth Special Deluxe Hot Rod, Automati...",0
135307,135307,wowcream.ru,Купить It's Skin Сыворотка питательная Power 1...,0


In [4]:
test_df = pd.read_csv("test.csv")
test_df.head()

,ID,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67


В дата сети данные идут в разнобой, их надо очистить. Обработаем их

In [5]:
class Preprocessor:
    '''
    Класс очистки раздела title
    param titles           : Вводимый массив
    param token            : Токен
    param texts            : 
    param lemmatizer_cache : кеш леммы
    param lemmatizer       : лемма
    param emot_handler     : массив эмоджи
    '''
    def __init__(self, titles, token_pattern=''):
        self.titles = titles
        self.token = RegexpTokenizer(token_pattern)
        self.texts = []
        self.lemmatizer_cache = {}
        self.lemmatizer = pymorphy3.MorphAnalyzer()
        #self.stemmer = SnowballStemmer(language='russian')
        self.emot_handler = emot.core.emot()

    def __cutEmoji(self):
        '''
        Вырежет все смайлы
        '''
        def replace_emoji(self, text):
            emojies_res = self.emot_handler.emoji(text)
            emoticons_res = self.emot_handler.emoticons(text)
            if emojies_res['flag']:
                values = emojies_res['value']
                means = emojies_res['mean']
                for i in range(len(values)):
                    text = text.replace(values[i], " " + means[i] + " ")
            if (emoticons_res['flag']):
                values = emoticons_res['value']
                means = emoticons_res['mean']
                for i in range(len(values)):
                    text = text.replace(values[i], " " + means[i] + " ")
            return text
        
        self.titles = [ replace_emoji(self, text) for text in progressbar(self.titles) ]

    def __tokenizeText(self):
        '''
        Токенизирует текст
        '''
        self.texts = [ self.token.tokenize(str(text).lower()) for text in self.titles ]


    # Бред выходит.
    #def __stemming(self):
    #    '''
    #    Стемминг — поиск общей основы (стемме) различных форм слова
    #    путём отбрасывания суффиксов, окончаний.
    #    * ending => end 
    #    * running => run
    #    (нужно не просто удалить ing !) 
    #    * sing => sing
    #    '''
    #    self.texts = [ self.stemmer.stem(text) for text in self.texts ] 

    def __lemmatize(self):
        '''
        Лемманизирует. Лемма - приведение слов к словарной форме (лемме):
        для существительного — единственное число, именительный падеж; 
        для глагола — инфинитив…
        * бегущий => бежать
        * люди => человек
        * рой => рыть
        * рой => рой
        '''
        def lemmatize_simple(self, token):
            if self.lemmatizer.word_is_known(token):
                if token not in self.lemmatizer_cache:
                    self.lemmatizer_cache[token] = self.lemmatizer.parse(token)[0].normal_form
                return self.lemmatizer_cache[token]
            return token
        self.texts = [ [ lemmatize_simple(self, token) for token in text ] for text in progressbar(self.texts) ]

    def __cutStopWordInText(self):
        '''
        Вырежет все стоп слова
        '''
        stopword_set = set(nltk.corpus.stopwords.words('russian') + nltk.corpus.stopwords.words('english'))
        self.texts = [ [ token for token in text if token not in stopword_set ] for text in self.texts ]

    def clearSomeTextData(self):
        self.__cutEmoji()
        self.__tokenizeText()
        self.__lemmatize()
        self.__cutStopWordInText()
        return [" ".join(text) for text in self.texts]

    def __tokenizeUrl(self):
        self.texts = [ re.split("-|\.", str(url)) for url in self.titles ]
    
    def __segment(self):
        self.texts = [ [ seg for part in url for seg in wordsegment.segment(part) ] for url in progressbar(self.texts) ]

    def __cutStopWordInUrl(self):
        stopword_set = { 'ru', 'com', 'www', 'md', 'org', 'online' }
        self.texts = [ [ token for token in url if token not in stopword_set ] for url in progressbar(self.texts) ]

    def clearSomeUrlData(self):
        self.__tokenizeUrl()
        self.__segment()
        self.__cutStopWordInUrl()
        return [ " ".join(text) for text in self.texts ]

In [6]:
def preprocess(df):
    df_new = df.copy()
    df_new["title"] = Preprocessor(df_new["title"].values, TOKEN_PATTERN).clearSomeTextData()
    df_new["url"] = Preprocessor(df_new["url"].values).clearSomeUrlData()
    return df_new

In [7]:
y_train = train_df["label"].astype(int).values
X_train = train_df.drop(["ID", "label"], axis=1)
X_test = test_df.drop(["ID"], axis=1)
X_train

,url,title
0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц..."
1,www.kp.by,Эта песня стала известна многим телезрителям б...
2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...
3,colorbox.spb.ru,Не Беси Меня Картинки
4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...
...,...,...
135304,mail.ru,пора тюльпанов турецкий сериал на русском язык...
135305,www.ntv.ru,Остросюжетный сериал «Шеф. Игра на повышение»....
135306,topclassiccarsforsale.com,"1941 Plymouth Special Deluxe Hot Rod, Automati..."
135307,wowcream.ru,Купить It's Skin Сыворотка питательная Power 1...


In [8]:
X_train_preprocessed = preprocess(X_train)
X_train_preprocessed

100% (135309 of 135309) |################| Elapsed Time: 0:00:17 Time:  0:00:170001
100% (135309 of 135309) |################| Elapsed Time: 0:00:13 Time:  0:00:130001
100% (135309 of 135309) |################| Elapsed Time: 0:14:40 Time:  0:14:400719
100% (135309 of 135309) |################| Elapsed Time: 0:00:00 Time:  0:00:000:00


,url,title
0,m kp,экс министр экономика молдова глава мидэи цель...
1,kp by,песня стать известный многий телезритель благо...
2,fan serials tv,банши сезон серия бремя красота смотреть онлайн
3,colorbox spb,бесить картинка
4,tula sport,новомосковск сыграть следж хоккеист алексински...
...,...,...
135304,mail,пора тюльпан турецкий сериал русский язык резу...
135305,ntv,остросюжетный сериал шеф игра повышение серия
135306,top classic cars for sale,plymouth special deluxe hot rod automatic smal...
135307,wow cream,купить skin сыворотка питательный power formul...


In [9]:
def getTopWords(df, count):
    '''
    Выведем топ слов
    df    : Почищенный дата сет
    count : Топ
    '''
    tag_words_counter = Counter([token for doc in df for token in doc.split()])
    return [i[0] for i in tag_words_counter.most_common(count)]

In [10]:
tagWordsTitle = getTopWords(X_train_preprocessed[y_train == 1].title.values, 50)
print(tagWordsTitle)

['порно', 'porn', 'видео', 'sex', 'videos', 'онлайн', 'hd', 'video', 'секс', 'com', 'смотреть', 'xxx', 'free', 'бесплатно', 'porno', 'фото', 'скачать', 'biqle', 'girls', 'девушка', 'movies', 'страница', 'page', 'big', 'daftsex', 'anal', 'download', 'русский', 'tube', 'perfect', 'голый', 'hot', 'mp', 'p', 'член', 'эротика', 'tits', 'teen', 'молодой', 'nude', 'sexy', 'ass', 'pussy', 'торрент', 'эротический', 'ru', 'girl', 'рассказ', 'красивый', 'парень']


In [11]:
tagWordsUrl = getTopWords(X_train_preprocessed[y_train == 1].url.values, 100)
print(tagWordsUrl)

['porno', 'net', 'porn', 'sex', 'x', 'tv', 'le', 'daft', 'biq', 'm', 'video', 'cc', 'tube', 'xxx', 'me', 'girls', 'mobi', 'info', 'perfect', 'videos', 'club', 'an', 'en', 'censored', 'pro', 'y', 'a', 'er', 'uk', 'hd', 'k', 'xyz', 'e', '24', 'top', 'i', 'comics', 'ero', 'co', 'tubs', 'exer', 'on', 'bobs', 'free', 'do', 'wap', 'rus', 'life', 'f', 'ch', 'just', 'to', 'kiss', 'raj', 'fap', 'jav', 'live', 'noodle', 'magazine', 'eu', 'xn', 'egg', 'pussy', 'hat', 'mail', 'biz', 'lib', 'site', 'space', 'sport', 'region', 'pening', 'ka', 'the', 'omni', 'scenta', 'gig', 'de', 'kino', 'reactor', 'seks', 'o', 'adult', 'bal', 'ovo', 'in', 'by', 'or', 'hub', 'jk', 'foto', 'vip', '24eropixel', 'union', 'sexy', 'embed', 'hot', 'ws', 'torrent', 'indian']


In [12]:
def markTagDF(df, tagTitle, tagUrl):
    '''
    Функция маркерует колличество таргетных слов
    '''
    taged_df = df.copy()
    tags = np.zeros(taged_df.shape[0])
    titles = taged_df['title'].values
    urls = taged_df['url'].values
    for i in range(taged_df.shape[0]):
        tags[i] = len([i for i in titles[i].split() if i in tagTitle] + [i for i in urls[i].split() if i in tagUrl])
    taged_df['tags_count'] = tags
    return taged_df

In [13]:
taged_X_train = markTagDF(X_train_preprocessed, tagWordsTitle, tagWordsUrl)
taged_X_train

,url,title,tags_count
0,m kp,экс министр экономика молдова глава мидэи цель...,1.0
1,kp by,песня стать известный многий телезритель благо...,1.0
2,fan serials tv,банши сезон серия бремя красота смотреть онлайн,3.0
3,colorbox spb,бесить картинка,0.0
4,tula sport,новомосковск сыграть следж хоккеист алексински...,1.0
...,...,...,...
135304,mail,пора тюльпан турецкий сериал русский язык резу...,3.0
135305,ntv,остросюжетный сериал шеф игра повышение серия,0.0
135306,top classic cars for sale,plymouth special deluxe hot rod automatic smal...,2.0
135307,wow cream,купить skin сыворотка питательный power formul...,0.0


In [14]:
new_X_tag_train, X_tag_val, new_y_tag_train, y_tag_val = train_test_split(taged_X_train, y_train, test_size=0.25, random_state=42)
new_X_train

,url,title,tags_count
109524,new inform,американский сми оценить азиатский разворот по...,0.0
13548,avatar ko,аватар девушка чужой аватар тигр фото прикол а...,2.0
62727,family guy fox fan tv,серия сезон мультсериал гриффины английский су...,4.0
114853,yoox,модель футляр женщина anna rachele yoox россия,0.0
60402,drive 2,бить руль торможение скорость км chevrolet ave...,0.0
...,...,...,...
110268,porno photo pro,смотреть бритый вагин фото,4.0
119879,multi listing su,продать трехкомнатную вторичку мкр олимпийский...,0.0
103694,digital 1k by,sharp dg купить минск k,1.0
131932,citi link,купить web камера logitech conferencecam rally...,0.0


**Наблюдение:** url не влият особо на выборку, так как если ссылка содержит 18+ контент, то title тоже его содержит

In [30]:
X_train_no_ut = taged_X_train.copy()
del X_train_no_ut['url']
del X_train_no_ut['tags_count']
X_train_no_ut

,title
0,экс министр экономика молдова глава мидэи цель...
1,песня стать известный многий телезритель благо...
2,банши сезон серия бремя красота смотреть онлайн
3,бесить картинка
4,новомосковск сыграть следж хоккеист алексински...
...,...
135304,пора тюльпан турецкий сериал русский язык резу...
135305,остросюжетный сериал шеф игра повышение серия
135306,plymouth special deluxe hot rod automatic smal...
135307,купить skin сыворотка питательный power formul...


In [31]:
X_train_no_ut.describe(include='all').T

,count,unique,top,freq
title,135309,126681,,802


## Выбор модели

Для начала создадим копию датасета

In [32]:
X_train_no = X_train_no_ut.copy()

In [37]:
X_train_no

,title
0,экс министр экономика молдова глава мидэи цель...
1,песня стать известный многий телезритель благо...
2,банши сезон серия бремя красота смотреть онлайн
3,бесить картинка
4,новомосковск сыграть следж хоккеист алексински...
...,...
135304,пора тюльпан турецкий сериал русский язык резу...
135305,остросюжетный сериал шеф игра повышение серия
135306,plymouth special deluxe hot rod automatic smal...
135307,купить skin сыворотка питательный power formul...


### LogisticRegression

#### LogisticRegression без гиперпараметров

In [213]:
X_tfidf_train = tfidf.fit_transform(X_train_no['title'])
y_tfidf_train = y_train.copy()

X_tf_train, X_tf_val, y_tf_train, y_tf_val = train_test_split(X_tfidf_train, y_tfidf_train, test_size=0.3, random_state=42)

In [212]:
X_tfidf_train.shape

(135309, 119402)

In [174]:
lr_model = LogisticRegression()
lr_model.fit(X_tf_train, y_tf_train)

LogisticRegression()

In [175]:
lr_y_train_pred = lr_model.predict(X_tf_train)
lr_y_test_pred = lr_model.predict(X_tf_val)

In [176]:
lr_f1_y_train = f1_score(y_tf_train, lr_y_train_pred)
lr_f1_y_test = f1_score(y_tf_val, lr_y_test_pred)
print(f'LogisticRegression train := {lr_f1_y_train}')
print(f'LogisticRegression test := {lr_f1_y_test}')

LogisticRegression train := 0.9337379410299379
LogisticRegression test := 0.910383455407152


#### LogisticRegression с гиперпаметрами

In [177]:
lr_grid_params = {
    'C' : np.logspace(-1, 1, 10),
    'penalty': ['l1', 'l2'],
    'max_iter': [10, 50, 100]
}
lr_grid_pipeline = pipeLineForTFIDF(LogisticRegression())

In [178]:
lr_grid = GridSearchCV(LogisticRegression(), lr_grid_params, cv=5, scoring='f1', n_jobs=-1)
lr_grid.fit(X_tf_train, y_tf_train) # Тут может выйти огромный красный блок
# Если вышел, значит все ок!!!

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([ 0.1       ,  0.16681005,  0.27825594,  0.46415888,  0.77426368,
        1.29154967,  2.15443469,  3.59381366,  5.9948425 , 10.        ]),
                         'max_iter': [10, 50, 100], 'penalty': ['l1', 'l2']},
             scoring='f1')

In [179]:
lr_best_penalty = lr_grid.best_params_['penalty']
lr_best_C = lr_grid.best_params_['C']
lr_best_max_iter = lr_grid.best_params_['max_iter']
lr_best_score = lr_grid.best_score_

In [180]:
print(f'Лучшие распределение: {"Гаусса" if lr_best_penalty == "l2" else "Лапласа"}')
print(f'Лучшая обратная величина регуляризации: {lr_best_C}')
print(f'Лучшие колличество итераций: {lr_best_max_iter}')
print(f'Лучшая LogisticRegression: {lr_best_score}')

Лучшие распределение: Гаусса
Лучшая обратная величина регуляризации: 10.0
Лучшие колличество итераций: 50
Лучшая LogisticRegression: 0.9438706163458764


In [181]:
lr_model_best = lr_grid.best_estimator_
lr_param_y_train_pred = lr_model_best.predict(X_tf_train)
lr_param_y_pred = lr_model_best.predict(X_tf_val)

In [182]:
lr_f1_param_y_train = f1_score(y_tf_train, lr_param_y_train_pred)
lr_f1_param_y_test = f1_score(y_tf_val, lr_param_y_pred)

In [183]:
print(f'LogisticRegressionParam train := {lr_f1_param_y_train}')
print(f'LogisticRegressionParam test := {lr_f1_param_y_test}')

LogisticRegressionParam train := 0.9826474533731446
LogisticRegressionParam test := 0.9456882255389718


### XGBoost

#### XGBoost без гиперпарметров

In [141]:
xgb_model = XGBClassifier()
xgb_model.fit(X_tf_train, y_tf_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [142]:
xgb_y_train_pred = xgb_model.predict(X_tf_train)
xgb_y_test_pred = xgb_model.predict(X_tf_val)

In [143]:
xgb_f1_y_train = f1_score(y_tf_train, xgb_y_train_pred)
xgb_f1_y_test = f1_score(y_tf_val, xgb_y_test_pred)
print(f'XGBClassifier train := {xgb_f1_y_train}')
print(f'XGBClassifier test := {xgb_f1_y_test}')

XGBClassifier train := 0.9364799493647995
XGBClassifier test := 0.9238449283058948


#### XGBClassifier c подбором гиперпараметров

In [131]:
xgb_grid_params = {
    'n_estimators': [100],
    'learning_rate': np.logspace(0.01, 0.1, 5),
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10]
}

In [132]:
xgb_grid = GridSearchCV(XGBClassifier(), xgb_grid_params, cv=5, scoring='f1', n_jobs=-1)
xgb_grid.fit(X_tf_train, y_tf_train)

/opt/homebrew/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': array([1.02329299, 1.07770525, 1.13501082, 1.19536353, 1.25892541]),
                         'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
                         'n_estimators': [100]},
             scoring='f1')

In [137]:
xgb_best_params = xgb_grid.best_params_
print(f"Best Hyperparameters: {xgb_best_params}")

Best Hyperparameters: {'learning_rate': 1.023292992280754, 'max_depth': 9, 'n_estimators': 100}


In [138]:
xgb_best_model = xgb_grid.best_estimator_

In [139]:
xgb_best_y_train_pred = xgb_best_model.predict(X_tf_train)
xgb_best_y_test_pred = xgb_best_model.predict(X_tf_val)

In [140]:
xgb_f1_y_train = f1_score(y_tf_train, xgb_best_y_train_pred)
xgb_f1_y_test = f1_score(y_tf_val, xgb_best_y_test_pred)
print(f'XGBClassifier train := {xgb_f1_y_train}')
print(f'XGBClassifier test := {xgb_f1_y_test}')

XGBClassifier train := 0.953238199780461
XGBClassifier test := 0.9242048740190004


Результаты получились хужи скорее всего из за плохо подобранных гипер параметров

### CatBoost

Как я понимаю, исходя из этой [статьи](https://habr.com/ru/companies/otus/articles/778714/) CatBoost и XGBoost +- одно и тоже

#### CatBoost без гиперпараметров

In [144]:
cat_model = CatBoostClassifier()
cat_model.fit(X_tf_train, y_tf_train)

Learning rate set to 0.071923
0:	learn: 0.6019219	total: 216ms	remaining: 3m 35s
1:	learn: 0.5268876	total: 358ms	remaining: 2m 58s
2:	learn: 0.4681890	total: 502ms	remaining: 2m 46s
3:	learn: 0.4174420	total: 643ms	remaining: 2m 40s
4:	learn: 0.3760495	total: 786ms	remaining: 2m 36s
5:	learn: 0.3435982	total: 928ms	remaining: 2m 33s
6:	learn: 0.3154508	total: 1.07s	remaining: 2m 31s
7:	learn: 0.2920142	total: 1.21s	remaining: 2m 30s
8:	learn: 0.2726388	total: 1.36s	remaining: 2m 29s
9:	learn: 0.2563691	total: 1.5s	remaining: 2m 28s
10:	learn: 0.2427332	total: 1.65s	remaining: 2m 27s
11:	learn: 0.2309556	total: 1.78s	remaining: 2m 26s
12:	learn: 0.2206820	total: 1.93s	remaining: 2m 26s
13:	learn: 0.2116809	total: 2.07s	remaining: 2m 25s
14:	learn: 0.2040360	total: 2.21s	remaining: 2m 24s
15:	learn: 0.1972733	total: 2.35s	remaining: 2m 24s
16:	learn: 0.1909405	total: 2.49s	remaining: 2m 24s
17:	learn: 0.1853635	total: 2.63s	remaining: 2m 23s
18:	learn: 0.1804585	total: 2.77s	remaining: 

In [145]:
cat_y_train_pred = cat_model.predict(X_tf_train)
cat_y_test_pred = cat_model.predict(X_tf_val)

In [147]:
cat_f1_y_train = f1_score(y_tf_train, cat_y_train_pred)
cat_f1_y_test = f1_score(y_tf_val, cat_y_test_pred)
print(f'CatBoostClassifier train := {cat_f1_y_train}')
print(f'CatBoostClassifier test := {cat_f1_y_test}')

CatBoostClassifier train := 0.9434825513591101
CatBoostClassifier test := 0.9238956891963811


В целом результат по **CatBoostClassifier** такой же примерно как и у **XGBClassifier**. Тогда подбор гиперпараметров осуществлять не буду.

### Промежуточный вывод

Из 3-ех моделей лучшей оказалась **LogisticRegression** по итогу ее применим для решения задачи.

## Решение задачи на основе обученной модели

In [152]:
X_test_df = preprocess(X_test)

100% (165378 of 165378) |################| Elapsed Time: 0:00:21 Time:  0:00:210001
100% (165378 of 165378) |################| Elapsed Time: 0:00:15 Time:  0:00:150001
100% (165378 of 165378) |################| Elapsed Time: 0:00:35 Time:  0:00:350002
100% (165378 of 165378) |################| Elapsed Time: 0:00:00 Time:  0:00:000:00


In [222]:
X_test_df.shape

(165378, 2)

In [223]:
X_test_tfidf = tfidf.transform(X_test_df['title'])

In [224]:
X_test_tfidf.shape

(165378, 119402)

In [225]:
test_df["label"] = lr_model_best.predict(X_test_tfidf)

In [226]:
test_df[["ID", "label"]].to_csv("hw3_result.csv", index=False)
!cat hw3_result.csv | head

ID,label
135309,0
135310,0
135311,0
135312,1
135313,0
135314,0
135315,0
135316,0
135317,0
cat: stdout: Broken pipe


# Вывод

*Public:* 0.9485443466486121

В общем это не предел, логистическую регрессию можно еще улучшить более глубоко поигравшись с параметрами, но технические ресурсы не позволяют(пишу по умолчанию нотебуки локально). Так же Cat- и XGB- Boosting тоже должны показывать результат лучше после подбора параметров